In [1]:
import pandas as pd 
from youtube_videos import youtube_search
import requests
import pickle as pk
from bs4 import BeautifulSoup as bs

In [2]:
categories=["Travel Blogs","Science and Technology","Food","Manufacturing","History","Art and Music"]

In [4]:
min_data=1700

def get_videos_links(categories):
    videos=[]
    for cat in categories[:]:
        k=0
        token=None
        while k<min_data:
            vid_data= youtube_search(cat,token=token)
            token=vid_data[0]
            if len(vid_data[1]) == 0:
                break
            k+=len(vid_data[1])

            for i in vid_data[1]:
                videos.append((i['id']['videoId'],cat))
        print(cat," total_videos: ",k)
    return videos

def extract_data(videos_links,min_data):
    file=[]
    k=0
    cat=""
    for vid in videos_links:
        if cat!= vid[1]:
            print()
            print(vid[1])
            cat=vid[1]    
            k=0
        url="https://www.youtube.com/watch?v="+vid[0]
        html=requests.get(url)
        data=bs(html.text,"html.parser")
        title=data.findAll("span",attrs={"class":"watch-title"})
        if len(title)==0:
            continue
        desc=raw_html=data.findAll("p",attrs={"id":"eow-description"})[0].text
        file.append([vid,title[0]['title'],desc,vid[1]])
        print(k,end=" ")
        k+=1
    return file

In [5]:
# video_links=get_videos_links(categories=categories)

# with open("video_links.pkl","wb") as file:
#     pk.dump(video_links,file)

In [7]:
with open("video_links.pkl","rb") as file:
    video_links=pk.load(file)

# data=extract_data(videos_links=video_links,min_data=min_data)

In [18]:
Dataset=pd.DataFrame(file)
Dataset.columns=["Video_id","Title","Description","Category"]
Dataset.head()

,Video_id,Title,Description,Category
0,i9E_Blai8vk,TRAVEL VLOG ∙ Welcome to Bali | PRISCILLA LEE,I had the chance to fly out to Bali with my wh...,Travel Blogs
1,e2NQE41J5eM,How do I travel so much ! How do I earn money!!,SUBSCRIBE - https://goo.gl/dEtSMJ (‘MountainTr...,Travel Blogs
2,ehmsJLZlCZ0,Ep 1| Travelling through North East India | Of...,"The journey to Arunachal, North East India beg...",Travel Blogs
3,-LzdIILq5vE,GOA TRAVEL DIARY | FOUR DAYS IN GOA | TRAVEL O...,Hope you enjoy MY GOA TRAVEL DIARY this video!...,Travel Blogs
4,7ByoBJYXU0k,5 Steps to Becoming a Travel Blogger,"Travel blogger, Nikki Vargas, of The Pin the M...",Travel Blogs


In [ ]:
Dataset.to_csv("Dataset_youtube.csv")